# Ratings 

In [3]:
import pymongo

# using mongo queries
questa potrebbe esser la nostra query complessa

In [10]:
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
db = client["movie_dataset"]

In [20]:
pipeline = [
    #{"$limit":1000},
    {"$set":{ # 1 if rating > 2.5 else -1
         "rating":{"$switch": { 
            "branches": [ 
                { 
                    "case": { "$lt": [ "$rating", 2.5 ] }, 
                    "then": -1
                },  
                { 
                    "case": { "$gte": [ "$rating", 2.5 ] }, 
                    "then": 1
                },  
                #"default":0 # senza default è null e di consegenza il campo non viene salvato -> save memory
            ],
         }},
        
    }},# _id,userId,MovieId,timestamp,rating: (-1 OR 1)
    {"$group":#group ratings by user in a suittable format to make a object in the next stage
        {"_id":"$userId",
        "ratings":
            {"$push":
                {"k":{"$toString":"$movieId"},"v":"$rating"}
            }
        }
    },#_id: (user_id), ratings: Array[{k:movieId,v:rating}, ...]
    {"$project":{ "_id":"$_id","ratings":{  "$arrayToObject":"$ratings" }}},
    #_id: (user_id), ratings: {movieId:rating, ...}]
    {"$set":{
        "ratings._id":"$_id"
    }}, #_id: (user_id), ratings: {_id:user_id, movieId:rating, ...}]
    {"$replaceRoot":{"newRoot":"$ratings"}},
    #_id:user_id, movieId:rating, ...
    {"$out":"reshaped_ratings"}
]
db.ratings.aggregate(pipeline[], allowDiskUse=True)
#THE SPEED IS INCOPARABLE 

# Using spark  TODO 
too slow

In [22]:
assert False, "TODO"

AssertionError: TODO

In [ ]:
import helper as h
import pyspark.sql.functions as F

In [4]:
r_spark = h.get_session("movie_dataset","ratings")

In [2]:
r_spark.conf.set('spark.sql.pivotMaxValues', '50000')
r_spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '-1')

In [ ]:
df = r_spark.read.format(h.FORMAT).option("pipeline",{"$limit":10000}).load()
df.createOrReplaceTempView("ratings")
#df.cache()
df.head()

In [5]:
#filter by top 2000 voted film
m_df = r_spark.read.format(h.FORMAT).option("pipeline",[
    {"$group":{"_id":"$movieId","count":{"$sum":1}}},#count occurence of rating for each movie
    { "$sort" : { "count" : -1}},# sort desc
    {"$limit":100}# take firsts 2k
]).load()
m_df.show()

+----+-----+
| _id|count|
+----+-----+
| 318|  210|
| 593|  200|
| 296|  198|
| 356|  198|
| 260|  193|
|2571|  191|
| 480|  171|
|   1|  167|
|1196|  159|
| 589|  157|
| 527|  155|
|1210|  155|
|1198|  154|
|  50|  150|
| 110|  145|
|4993|  144|
|2959|  144|
| 858|  142|
| 780|  139|
|1270|  139|
+----+-----+
only showing top 20 rows



In [6]:
#random split of users (first 10k)
u_df = r_spark.read.format(h.FORMAT).option("pipeline",[
    {"$group":{"_id":"$userId"}},
    {"$limit":20}# take firsts 10k
]).load()
u_df.show()

+---+
|_id|
+---+
|129|
|117|
|204|
|104|
| 90|
|187|
| 11|
| 92|
|135|
|221|
|182|
| 97|
| 67|
|220|
|114|
|  1|
|  2|
|179|
|159|
| 70|
+---+
only showing top 20 rows



In [7]:
df = df.join(m_df, df.movieId.isin(m_df._id)).select('userId','movieID','rating')
df = df.join(u_df, df.userId.isin(u_df._id)).select('userId','movieID','rating')
#max size 10k*2k
df.show()

+------+-------+------+
|userId|movieID|rating|
+------+-------+------+
|    11|     32|   3.5|
|    11|     34|   4.0|
|    11|     47|   3.5|
|    11|    110|   3.5|
|    11|    165|   3.5|
|    11|    231|   2.5|
|    11|    260|   3.0|
|    11|    296|   4.0|
|    11|    318|   4.5|
|    11|    344|   3.0|
|    11|    356|   4.0|
|    11|    364|   3.0|
|    11|    457|   3.5|
|    11|    480|   3.5|
|    11|    527|   4.5|
|    11|    593|   4.0|
|    11|    736|   3.0|
|    11|   1089|   3.5|
|    11|   1265|   2.5|
|    11|   1270|   3.0|
+------+-------+------+
only showing top 20 rows



In [ ]:
piv = df.groupBy(df.userId).pivot(df.movieId).agg(df.rating)

In [ ]:
piv.head()

In [9]:
df.collect()

KeyboardInterrupt: 